# Container on Google Cloud Platform

This notebook explains how to package and deploy this repository as a container on Google Cloud Platform (Artifact Registry + Cloud Run) and why you might want to do that for renewable-energy forecasting applications.

- Short purpose: provide reproducible runtime, enable easy scaling, and integrate with CI/CD for rapid updates.


## Prerequisites
- VS Code with the Remote - Containers extension
- Docker Desktop
- Google Cloud SDK (`gcloud`) installed locally if you plan to run commands from your machine

## Develop in a VS Code devcontainer
1. Open the project in VS Code and ensure the Remote - Containers extension is installed.
2. Reopen in Container. VS Code will load the devcontainer and install dependencies.

Inside the devcontainer the FastAPI application (or equivalent) will be ready for development with dependencies pre-installed.

## 2. Why deploy to the cloud

Deploying this application (or model pipeline) to Cloud Run / managed infrastructure is useful because:

- Reproducibility: containers ensure the same environment across development, CI, and production.
- Scalability: Cloud Run can scale the service to handle bursts of requests (e.g., many forecast queries).
- Automation: CI/CD pipelines (Cloud Build or GitHub Actions) let you automatically build and deploy on changes.
- Integration: cloud services make it easier to connect to storage (Cloud Storage), artifact registries, monitoring and logging.

Typical use-cases for deploying these notebooks/code:
- Serving forecast-derived indicators (e.g., solar/wind generation estimates) via an HTTP API to downstream dashboard/clients.
- Batch processing pipelines that run on schedule and store processed results in cloud storage.
- A lightweight web service for demonstration/training for stakeholders.


### 1. Authenticate with Google Cloud
Run the following to initialize gcloud and choose your project/region:

In [ ]:
# Initialize gcloud and choose project/region
gcloud init

### 2. Create the Deployment and App Service Accounts
Create two service accounts: one for deploying (DEPLOY_SERVICE_ACCOUNT) and one for the Cloud Run service runtime (APP_SERVICE_ACCOUNT).
Set environment variables (replace YOUR_PROJECT_ID):

In [ ]:
export PROJECT_ID="YOUR_PROJECT_ID"
export DEPLOY_SERVICE_NAME="cloud-run-deployer"
export DEPLOY_SERVICE_ACCOUNT="${DEPLOY_SERVICE_NAME}@${PROJECT_ID}.iam.gserviceaccount.com"
export APP_SERVICE_NAME="cloud-run-runtime"
export APP_SERVICE_ACCOUNT="${APP_SERVICE_NAME}@${PROJECT_ID}.iam.gserviceaccount.com"

In [ ]:
# Create deploy service account
gcloud iam service-accounts create "${DEPLOY_SERVICE_NAME}" \
    --description="Service account for deploying to Cloud Run" \
    --display-name="Cloud Run Deployer" \
    --project="${PROJECT_ID}"

Grant roles to the deployer service account if missing (example roles shown):
- roles/run.admin
- roles/artifactregistry.repoAdmin
- roles/cloudbuild.builds.builder
- roles/viewer

In [ ]:
gcloud projects add-iam-policy-binding "${PROJECT_ID}" \
  --member="serviceAccount:${DEPLOY_SERVICE_ACCOUNT}" \
  --role="roles/run.admin"

gcloud projects add-iam-policy-binding "${PROJECT_ID}" \
  --member="serviceAccount:${DEPLOY_SERVICE_ACCOUNT}" \
  --role="roles/artifactregistry.repoAdmin"

Create the app runtime service account and grant it the viewer role for Firebase if needed:

In [ ]:
gcloud iam service-accounts create ${APP_SERVICE_NAME} \
  --description="Service account for a Cloud Run service" \
  --display-name="Cloud Run Runtime" \
  --project=${PROJECT_ID}

gcloud projects add-iam-policy-binding "${PROJECT_ID}" \
  --member="serviceAccount:${APP_SERVICE_ACCOUNT}" \
  --role="roles/firebaseauth.viewer"

Grant DEPLOY_SERVICE_ACCOUNT the Service Account User role so it can set the runtime identity when deploying:

In [ ]:
gcloud iam service-accounts add-iam-policy-binding ${APP_SERVICE_ACCOUNT} \
  --member="serviceAccount:${DEPLOY_SERVICE_ACCOUNT}" \
  --role="roles/iam.serviceAccountUser"

Generate a service account key for the deployer and store it as a GitHub Actions secret (`GCP_SA_KEY`).
**Warning**: Treat service account keys as sensitive secrets. Add the JSON content to GitHub repository secrets (Actions -> Secrets).

In [ ]:
gcloud iam service-accounts keys create cloud-run-deployer-key.json \
  --iam-account=${DEPLOY_SERVICE_ACCOUNT} \
  --project=${PROJECT_ID}

### Enable Required APIs
Check and enable Artifact Registry and Cloud Run APIs if necessary:

In [ ]:
gcloud services list --enabled --filter="config.name:artifactregistry.googleapis.com OR config.name:run.googleapis.com"

gcloud services enable artifactregistry.googleapis.com
gcloud services enable run.googleapis.com

### Artifact Registry Setup
Create an Artifact Registry repository for Docker images:

In [ ]:
gcloud artifacts repositories create container-name \
  --repository-format=docker \
  --location=europe-west4 \
  --description="Docker image for application"

### Build and Deploy (Cloud Build + Cloud Run)
Use `cloudbuild.yaml` to build, test, and push the image, then deploy to Cloud Run. Example commands:

In [ ]:
# Submit build (Cloud Build will build and push to Artifact Registry)
gcloud builds submit --config=cloudbuild.yaml --project ${PROJECT_ID}

# Deploy/replace a Cloud Run service using a service manifest (service.yaml)
gcloud run services replace service.yaml --region europe-west4 --project ${PROJECT_ID}

### CI/CD - GitHub Actions
Create `.github/workflows/deploy.yml` to build, push, and deploy on push to `main`. The workflow needs the `GCP_SA_KEY` secret containing the deployer service account JSON key.

## 5. Files added in this repo (examples)

I added example deployment artifacts to this repository to help you get started:

- `Dockerfile.prod` — production-ready Dockerfile (Gunicorn + Uvicorn)
- `cloudbuild.yaml` — Cloud Build config to build/push image and deploy to Cloud Run
- `.github/workflows/deploy.yml` — GitHub Actions workflow using the `GCP_SA_KEY` secret
- `README_DEPLOY.md` — short operator guide

Open these files to adapt them for your project. The `Dockerfile.prod` expects a `requirements.txt` in the repo root; if you use another dependency manager update accordingly.


### Example: `Dockerfile.prod` (excerpt)

```dockerfile
# Example production Dockerfile (see Dockerfile.prod in repo root)
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt ./
RUN pip install --upgrade pip && pip install -r requirements.txt
COPY . /app
ENV PORT=8080
EXPOSE 8080
CMD ["gunicorn", "-k", "uvicorn.workers.UvicornWorker", "main:app", "-b", "0.0.0.0:8080", "--workers", "4"]
```